<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE__CNN_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import joblib
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
CONFIG = {
    'BASE_DIR_AF'      : '/content/drive/MyDrive/Research/Crowd Emotion v4/',
    'BASE_DIR_VAL'     : '/content/drive/MyDrive/Research/Crowd Emotion Val/',
    'WINDOWS_DIR'      : 'Windows/',
    'DATASET_DIR'      : 'Dataset_STFT/',
    'LOG_DIR'          : 'Logs/',
    'FOLDS'            : ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5'],
    'EMOTIONS'         : ['Approval', 'Disapproval', 'Neutral'],

    'DATASET_SHAPE'    : (14882, 219, 201),

    'FRAME_LEN'        : 400,
    'FRAME_STEP'       : 200,
    'FFT_LEN'          : 400,

    'STFT_CONFIG'      : 'FL_400_FS_200_FFT_400',

    'TEST_FOLD'        : 'Fold 1',
    'EPOCHS'           : 10,
    'LEARNING_RATE'    : 0.001,
}

In [3]:
# count = 0

# X = np.zeros(CONFIG['DATASET_SHAPE'], dtype='float16')
# y = np.zeros((CONFIG['DATASET_SHAPE'][0], 1), dtype='uint8')
# f = np.zeros((CONFIG['DATASET_SHAPE'][0], ), dtype='uint8') 
# c = np.zeros((CONFIG['DATASET_SHAPE'][0], ), dtype='uint8') 

# for emotion in CONFIG['EMOTIONS']:
#     print('processing audio for ' + emotion, end=' ... ')
#     for fold in CONFIG['FOLDS']:
#         fold_path = os.path.join(
#             CONFIG['BASE_DIR_AF'],
#             CONFIG['WINDOWS_DIR'],
#             emotion,
#             fold
#         )
#         filenames = os.listdir(fold_path)
#         for filename in filenames:
#             file_path = os.path.join(fold_path, filename)
#             data = joblib.load(file_path)

#             for i in range(data.shape[0]):
#                 x = data[i, :]
#                 spectrogram = tf.signal.stft(
#                    signals        = x,
#                    frame_length   = CONFIG['FRAME_LEN'],
#                    frame_step     = CONFIG['FRAME_STEP'],
#                    fft_length     = CONFIG['FFT_LEN'],
#                    window_fn      = tf.signal.hamming_window
#                 )
                
#                 magnitude = tf.abs(spectrogram).numpy()

#                 X[count, :] = magnitude
#                 y[count, :] = CONFIG['EMOTIONS'].index(emotion)
#                 f[count] = CONFIG['FOLDS'].index(fold)
#                 c[count] = int(filename[7:11])

#                 count += 1

#     print('√')

In [4]:
# joblib.dump(X, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_X.joblib'))
# joblib.dump(y, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_y.joblib'))
# joblib.dump(f, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_f.joblib'))
# joblib.dump(c, os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_c.joblib'))

In [5]:
X = joblib.load(os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_X.joblib'))
y = joblib.load(os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_y.joblib'))
f = joblib.load(os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_f.joblib'))
c = joblib.load(os.path.join(CONFIG['BASE_DIR_AF'], CONFIG['DATASET_DIR'], CONFIG['STFT_CONFIG'] + '_c.joblib'))

In [6]:
mask      = (f == CONFIG['FOLDS'].index(CONFIG['TEST_FOLD']))
train_X   = X[~mask, :]
train_y   = y[~mask, :]
val_X     = X[mask, :]
val_y     = y[mask, :]

In [7]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(X)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(219, 201, 1)),
    tf.keras.layers.experimental.preprocessing.Resizing(32, 32), 
    norm_layer,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 32, 32, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 32, 32, 201)       403       
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        57920     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [10]:
history = model.fit(
    train_X,
    train_y, 
    validation_data=(val_X, val_y),  
    epochs=100,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Epoch 1/100
375/375 [==============================] - 8s 13ms/step - loss: 83.1316 - accuracy: 0.4294 - val_loss: 22.1101 - val_accuracy: 0.4124
Epoch 2/100
375/375 [==============================] - 4s 9ms/step - loss: 58.8473 - accuracy: 0.3996 - val_loss: 15.6122 - val_accuracy: 0.4544
Epoch 3/100
375/375 [==============================] - 4s 9ms/step - loss: 43.0170 - accuracy: 0.4180 - val_loss: 11.7018 - val_accuracy: 0.4967
Epoch 4/100
375/375 [==============================] - 4s 9ms/step - loss: 31.4808 - accuracy: 0.4442 - val_loss: 8.0894 - val_accuracy: 0.5272
Epoch 5/100
375/375 [==============================] - 4s 9ms/step - loss: 22.3198 - accuracy: 0.4619 - val_loss: 6.4781 - val_accuracy: 0.5428
Epoch 6/100
375/375 [==============================] - 4s 9ms/step - loss: 18.2160 - accuracy: 0.4792 - val_loss: 5.0044 - val_accuracy: 0.5709
Epoch 7/100
375/375 [==============================] - 4s 9ms/step - loss: 14.2851 - accuracy: 0.4788 - val_loss: 3.9129 - val_accur